In [24]:
import pydeck as pdk
import pandas as pd
import json
import time


def convertMonthYearToMonthYear(monthYear):
    month = (monthYear % 12) + 1
    year = (monthYear // 12) + 2016
    return month, year

def convertMonthAndYeartoMonthYear(month, year):
    monthYear = (year - 2016) * 12 + month - 1
    return monthYear

def count_entries_within_radius(
    target_lon, target_lat, radius_miles, currYear, currMonth
):
    # Convert miles to degrees (roughly, considering Earth's radius)
    degrees_per_mile = 1 / 69  # Approximately
    radius_deg = radius_miles * degrees_per_mile

    # Calculate latitude and longitude bounds for the square around the target point
    min_lon = target_lon - radius_deg
    max_lon = target_lon + radius_deg
    min_lat = target_lat - radius_deg
    max_lat = target_lat + radius_deg

    # Filter the DataFrame to get entries within the specified latitude and longitude bounds
    filtered_df = df[
        (df["lon"] >= min_lon)
        & (df["lon"] <= max_lon)
        & (df["lat"] >= min_lat)
        & (df["lat"] <= max_lat)
        & (df["year"] == currYear)
        & (df["month"] == currMonth)
    ]

    # Get the count of entries within the radius
    count_within_radius = len(filtered_df)

    return count_within_radius


def monthToNum(shortMonth):
    return {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12,
    }[shortMonth]


def numTofullMonthName(num):
    return {
        1: "January",
        2: "Febuary",
        3: "March",
        4: "April",
        5: "May",
        6: "June",
        7: "July",
        8: "Augest",
        9: "September",
        10: "October",
        11: "November",
        12: "December",
    }[num]





In [25]:
weather_data_file_path = "../weather_data.json"

print("Loading weather data from", weather_data_file_path)

start = time.time()
weather_data = json.load(open(weather_data_file_path))
# end the timer
end = time.time()
print("Finished weather data from", weather_data_file_path)
# print the time it took to load the data in seconds to 2 decimal places
print("Time to load data:", round(end - start, 2), "seconds")

Loading weather data from ../weather_data.json
Finished weather data from ../weather_data.json
Time to load data: 0.23 seconds


In [26]:
data_list = []

print("loading data into data_list")
for key, value in weather_data.items():
    year = value["date"].split(", ")[2]
    month = value["date"].split(", ")[0].split(" ")[0]

    if "lon" in value and year in "20162017201820192020202120222023":
        data_list.append(
            {
                "lon": value["lon"],
                "lat": value["lat"],
                "temp": value["data"][0]["temp"],
                "year": int(year),
                "month": monthToNum(month),
                "monthYear": convertMonthAndYeartoMonthYear(monthToNum(month),int(year))
            }
        )
print("Finished loading data into data_list")
# Create a DataFrame from the filtered data
df = pd.DataFrame(data_list)

df = df.sort_values("monthYear")


loading data into data_list
Finished loading data into data_list


In [27]:
df["density"] = 0
# do df itterorws
print("Starting to calculate density")

for index, row in df.iterrows():
    # update the rows density
    df.at[index, "density"] = count_entries_within_radius(
        row["lon"], row["lat"], 5, row["year"], row["month"]
    )
print("Finished calculating density")

# print the first 10 rows density in a human way
for index, row in df.iterrows():
    if index == 15:
        break
    print(
        "In ",
        numTofullMonthName(row["month"]),
        " ",
        row["year"],
        " there were ",
        row["density"],
        " entries within 5 miles of ",
        row["lon"],
        ", ",
        row["lat"],
    )

mean_lat = df["lat"].mean()
mean_lon = df["lon"].mean()

Starting to calculate density
Finished calculating density
In  January   2016.0  there were  4.0  entries within 5 miles of  -81.3633 ,  36.8499
In  January   2016.0  there were  3.0  entries within 5 miles of  -81.9861 ,  36.5091
In  January   2016.0  there were  4.0  entries within 5 miles of  -84.1364 ,  34.6658
In  January   2016.0  there were  1.0  entries within 5 miles of  -72.9548 ,  43.3988
In  January   2016.0  there were  1.0  entries within 5 miles of  -73.1376 ,  42.7718
In  January   2016.0  there were  1.0  entries within 5 miles of  -72.0268 ,  43.8762
In  January   2016.0  there were  1.0  entries within 5 miles of  -75.9588 ,  40.6328
In  January   2016.0  there were  3.0  entries within 5 miles of  -74.7531 ,  41.1994
In  January   2016.0  there were  3.0  entries within 5 miles of  -77.4863 ,  39.9322
In  January   2016.0  there were  3.0  entries within 5 miles of  -73.2502 ,  42.2211
In  January   2016.0  there were  1.0  entries within 5 miles of  -74.5163 ,  41.

In [72]:
def display_map_for_month_year(month_year):
    subset_df = df[df["monthYear"] == month_year].copy()

    layer = pdk.Layer(
    "GridLayer",
    data=subset_df,
    pickable=True,
    extruded=True,
    cellSize=8046,
    tooltip=True,
    get_color="[0, temp, 0, 160]",
    elevationScale=400,
    get_position=["lon", "lat"],
    get_elevation="density",
)

    view_state = pdk.ViewState(latitude=df["lat"].mean(), longitude=df["lon"].mean(), zoom=4, pitch=45)

    # Create a deck
    deck = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={
        'html': '<b>Number of Hikers in a 5-Mile Radius:</b> {elevationValue}',
        'style': {
            'color': 'white'
        }
    }
)
    
    return deck.to_html("density.html" + str(month_year) + ".html")



In [73]:

full_layer = pdk.Layer(
"GridLayer",
data=df,
pickable=True,
extruded=True,
cellSize=20000,
tooltip={"text": "Density of entries: {density}"},
get_color="[0, temp, 0, 160]",
elevationScale=400,
get_position=["lon", "lat"],
# get_elevation="density",
)
full_view_state = pdk.ViewState(latitude=df["lat"].mean(), longitude=df["lon"].mean(), zoom=4, pitch=45)
# Create a deck
full_deck = pdk.Deck(layers=[full_layer], initial_view_state=full_view_state,  tooltip={
        'html': '<b>Number of Hikers in a 5-Mile Radius:</b> {elevationValue}',
        'style': {
            'color': 'white'
        }
    })

In [74]:
# print unique years, months, and denisty
print("Unique years:", df["year"].unique())
print("Unique months:", df["month"].unique().sort())
print("Unique density:", df["density"].unique())

Unique years: [2016 2017 2018 2019 2020 2021 2022]
Unique months: None
Unique density: [ 4  3  1  2  5 11 12  6 10  7  8  9 19 13 17 15 16 20 36 32 14 42 51 18
 38 56 22 28 23 26 76 86 60 27 41 46 37 30 25 24 58 97 63 31 33 57 39 48
 47 21 53 65 91 34 29 54 35 49 52 72 45 59 44 43 77 50]


In [76]:
currMonthYear = 16

print(convertMonthYearToMonthYear(currMonthYear))
display(display_map_for_month_year(currMonthYear))



(5, 2017)


Map for the 5th month aka May 2016